In [1]:
%cd ..

/home/umenty/dev/hackathon/Noimu


In [2]:
from soundAI.rl_model import RLModel
model = RLModel([["delightful", "calm", "depressing"], ["jazz", "rock"]], replay_buffer_size=1)
model

/home/umenty/miniconda3/envs/geek2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
all_inputs = model.network.generate_all_input_combinations(debug=True)
all_inputs

category_onehot_list
[((1, 0, 0), (0, 1, 0), (0, 0, 1)), ((1, 0), (0, 1))]
all_input_combinations
tensor([[1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1.]])


tensor([[1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1.]])

In [4]:
for input in all_inputs:
    print(
        model.catman.prompt_to_action_id(
            model.catman.combination_tensor_to_text(
                input
            ), model.network
        )
    )

0
1
2
3
4
5


In [5]:
prompt, wav = model.choose_music(debug=True)
prompt, wav

各行動のQ値は以下のようになります。
{'calm jazz.': 0.11002540588378906,
 'calm rock.': -0.0396575927734375,
 'delightful jazz.': 0.22696685791015625,
 'delightful rock.': -0.06619644165039062,
 'depressing jazz.': 0.3436603546142578,
 'depressing rock.': 0.16346168518066406}
ソフトマックス行動選択(t=0.5)の確率は以下のようになりました。
{'calm jazz.': 0.1558560454797911,
 'calm rock.': 0.11553422408804452,
 'delightful jazz.': 0.1969235731453789,
 'delightful rock.': 0.10956183584347569,
 'depressing jazz.': 0.24868889392440857,
 'depressing rock.': 0.1734354275189013}
depressing jazz. の音楽を選択しました


('depressing jazz.',
 tensor([[0.0074, 0.0059, 0.0089,  ..., 0.0271, 0.0292, 0.0312]],
        device='cuda:0'))

In [6]:
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write
import os

audio_write(os.path.splitext("./suzuki_test/test1.wav")[0], wav.cpu(), 32000, strategy="loudness")
display_audio(wav, sample_rate=32000)

CLIPPING ./suzuki_test/test1 happening with proba (a bit of clipping is okay): 0.001390625024214387 maximum scale:  1.2203342914581299


- 学習のテスト

In [7]:
model.add_latest_prompt('depressing jazz.')
model.add_feedback(50)
model.learn()

outputs tensor([[29.6563]], grad_fn=<StackBackward0>)
targets tensor([[50]])


In [8]:
prompt, wav = model.choose_music(debug=True)
prompt, wav

各行動のQ値は以下のようになります。
{'calm jazz.': -1.9283218383789062,
 'calm rock.': -1.5322284698486328,
 'delightful jazz.': -1.6490554809570312,
 'delightful rock.': -1.2994861602783203,
 'depressing jazz.': -1.999074935913086,
 'depressing rock.': -1.5527915954589844}
ソフトマックス行動選択(t=0.5)の確率は以下のようになりました。
{'calm jazz.': 0.08725361470976939,
 'calm rock.': 0.1926751724363384,
 'delightful jazz.': 0.15252853669438823,
 'delightful rock.': 0.3068902972453881,
 'depressing jazz.': 0.07574048261838455,
 'depressing rock.': 0.1849118962957313}
calm rock. の音楽を選択しました


('calm rock.',
 tensor([[0.0148, 0.0136, 0.0190,  ..., 0.1319, 0.1299, 0.1450]],
        device='cuda:0'))

In [9]:
audio_write(os.path.splitext("./suzuki_test/test2.wav")[0], wav.cpu(), 32000, strategy="loudness")
display_audio(wav, sample_rate=32000)

CLIPPING ./suzuki_test/test2 happening with proba (a bit of clipping is okay): 0.000375000003259629 maximum scale:  1.2402498722076416


In [10]:
model.add_latest_prompt('calm rock.')
model.add_feedback(20)
model.learn()

outputs tensor([[31.5322]], grad_fn=<StackBackward0>)
targets tensor([[20]])


In [11]:
prompt, wav = model.choose_music(debug=True)
prompt, wav

各行動のQ値は以下のようになります。
{'calm jazz.': -1.3689308166503906,
 'calm rock.': -0.0025234222412109375,
 'delightful jazz.': -1.3714733123779297,
 'delightful rock.': 0.029775619506835938,
 'depressing jazz.': -2.0726585388183594,
 'depressing rock.': -0.7689685821533203}
ソフトマックス行動選択(t=0.5)の確率は以下のようになりました。
{'calm jazz.': 0.02678249316478524,
 'calm rock.': 0.4118104714524027,
 'delightful jazz.': 0.026646650089045605,
 'delightful rock.': 0.439290667291569,
 'depressing jazz.': 0.006555425252316203,
 'depressing rock.': 0.08891429274988129}
delightful rock. の音楽を選択しました


('delightful rock.',
 tensor([[ 0.0125,  0.0080,  0.0551,  ..., -0.0161, -0.0325,  0.0050]],
        device='cuda:0'))

: 